In [1]:
from model import DualAttentionSeq2Seq
import torch
from torch.autograd import Variable
from sgdr import SGDRScheduler
from tqdm import tqdm
import numpy as np

In [2]:
FEATURE_DIR = '../feature_sets/'

In [3]:
X_train = np.load(FEATURE_DIR+'X_train.npy')
y_train = np.load(FEATURE_DIR+'y_train.npy')

X_valid = np.load(FEATURE_DIR+'X_valid.npy')
y_valid = np.load(FEATURE_DIR+'y_train.npy')

In [4]:
epoches = 1000

In [5]:

# import torch
# import numpy as np
# import torch.utils.data as utils

# my_x = [np.array([[1.0,2],[3,4]]),np.array([[5.,6],[7,8]])] # a list of numpy arrays
# my_y = [np.array([4.]), np.array([2.])] # another list of numpy arrays (targets)

# tensor_x = torch.stack([torch.Tensor(i) for i in my_x]) # transform to torch tensors
# tensor_y = torch.stack([torch.Tensor(i) for i in my_y])

# my_dataset = utils.TensorDataset(tensor_x,tensor_y) # create your datset
# my_dataloader = utils.DataLoader(my_dataset)

In [6]:
from sklearn.preprocessing import StandardScaler
x_scaler = StandardScaler()
x_scaler.fit(X_train)
X_train = x_scaler.transform(X_train)
X_valid = x_scaler.transform(X_valid)

y_scaler = StandardScaler()
y_scaler.fit(y_train)
y_train = y_scaler.transform(y_train)
y_valid = y_scaler.transform(y_valid)

In [7]:
# for run in range(1):
batch_size = 64
input_dims = X_train.shape[1]
sequence_length = 16
cell_size = 2
encoded_cell_size = 1

# Learning Rates
max_rate = 0.18
min_rate = 0.04
steps_per_cycle = 20000
warmup = 100


# setup model
model = DualAttentionSeq2Seq(input_dims, sequence_length, cell_size, encoded_cell_size)

criterion = torch.nn.MSELoss()
optimiser = torch.optim.SGD(model.parameters(), lr=max_rate)
scheduler = SGDRScheduler(optimiser, min_rate, max_rate, steps_per_cycle, warmup, 0)

# around 20 - 40 k epochs for training
for epoch in tqdm(range(1)):
    for features, label in zip(X_train, y_train[:,0]):
        input = Variable(torch.from_numpy(np.reshape(features, (-1,1,1))))
        target = Variable(torch.from_numpy(np.reshape(label, (-1,1,1))))
        optimiser.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimiser.step()
        scheduler.step()

    input = Variable(X_valid)
    target = Variable(y_valid)
    output = model(input)
    loss = criterion(output, target)

    if epoch % 200 == 0:
        print(loss)


  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: Expected object of scalar type Float but got scalar type Double for argument #4 'mat1'